Solver = cvxpy y la red neuronal tiene el solver incluido en la capa de salida

# Parte 1: obtenemos los datos

In [1]:
import numpy as np
import deep_inv_opt as io
import deep_inv_opt.plot as iop
import torch

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.max_open_warning'] = 0  # Let the plots flow!
%matplotlib inline

In [2]:
u_train = io.tensor(np.linspace(-1.5, 1.5, 4).reshape((-1, 1)))
u_train

tensor([[-1.5000],
        [-0.5000],
        [ 0.5000],
        [ 1.5000]], dtype=torch.float64)

In [3]:
u_val = io.tensor(np.linspace(-1.5, 1.5, 4).reshape((-1, 1)))
u_val

tensor([[-1.5000],
        [-0.5000],
        [ 0.5000],
        [ 1.5000]], dtype=torch.float64)

ahora generamos los x correspondientes del modelo real

In [4]:
class ExamplePLP(io.ParametricLP):
    # Generate an LP from a given feature vector u and weight vector w.
    def generate(self, u, w):
        c = [[torch.cos(w[0] + w[1]*u)],
             [torch.sin(w[0] + w[1]*u)]]

        A_ub = [[-1.0,  0.0],
                [ 0.0, -1.0],
                [ w[0], 1 + w[1]*u/3]]

        b_ub = [[ 0.2*w[0]*u],
                [-0.2*w[1]*u],
                [ w[0] + 0.1*u]]
        
        return c, A_ub, b_ub, None, None

In [5]:
plp_true = ExamplePLP(weights=[1.0, 1.0])

# Generate training targets by solve the true PLP at each u value.
x_train = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_train])
torch.save(x_train, "x_train.pt")

x_val = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_val])
torch.save(x_val, "x_val.pt")

In [6]:
x_val = torch.load("x_val.pt")
x_val

tensor([[ 0.3000,  1.1000],
        [ 0.1000, -0.1000],
        [-0.1000,  0.1000],
        [ 0.7000,  0.3000]], dtype=torch.float64)

In [7]:
x_train = torch.load("x_train.pt")
x_train

tensor([[ 0.3000,  1.1000],
        [ 0.1000, -0.1000],
        [-0.1000,  0.1000],
        [ 0.7000,  0.3000]], dtype=torch.float64)

# Parte 2: definimos la red y la entrenamos

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [9]:
# ver si estoy usando GPU
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
learning_rate = 1e-3
batch_size = 2

In [11]:
# definimos el dataset
class UDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data.clone().to(dtype=torch.float32) # nota: esta dando el warning porque estoy convirtiendo un tensor a otro, en ese caso es mejor usar clone()
        # si los datos de entrada no los voy a dar como un tensor, entonces hay que poner lo que he puesto: self.data = torch.tensor(data, dtype=torch.float32), self.targets = torch.tensor(targets, dtype=torch.float32)
        self.targets = targets.clone().to(dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Dataset con pares (u, x)
dataset = UDataset(u_train, x_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [12]:
val_dataset = UDataset(u_val, x_val)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [13]:
# funcion que resuelve el problema de programacion lineal (por ahora nos la creemos, pero hay que revisarla)
# hay que tener en cuenta que esta funcion debe realizarse con operaciones de pytorch para
# preservar el grafo de computo para poder hacer backpropagation

# def smooth_lp(c, A, b):
#     # Inicializar x con gradientes habilitados
#     x = torch.zeros(A.shape[1], requires_grad=True)
#     optimizer = torch.optim.SGD([x], lr=1e-3)

#     for _ in range(1000):
#         optimizer.zero_grad()
#         constraint_penalty = torch.sum(torch.relu(A @ x - b))
#         objective = torch.dot(c, x) + 1000.0 * constraint_penalty
#         objective.backward(retain_graph=True)  # Mantén el grafo activo
#         optimizer.step()
#     return x  # Sin detach()


In [14]:
# scipy descartado porque no funciona
# voy a probar con cvxpy
from diffcp import SolverError
import torch
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer

# def solver(c_, A_, b_):
#     m,n = A_.shape

#     A = cp.Parameter((m, n))
#     b = cp.Parameter(m)
#     c = cp.Parameter(n)
#     x = cp.Variable(n)

#     obj = cp.Minimize(c.T @ x)
#     cons = [ A @ x <= b ]
#     prob = cp.Problem(obj, cons)

#     layer = CvxpyLayer(prob, parameters=[c,A,b], variables=[x])

#     # solution, = layer(c_, A_, b_)
#     # return solution
#     try:
#         solution, = layer(c_, A_, b_)
#         return solution
#     except Exception as e:
#         # print("El problema no tiene solución o es numéricamente inestable.")
#         return torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=True)

# def smooth_lp(c, A, b):
#     ''' 
#     metodo de la M grande pero con descenso 
#     por el grandiente
#     '''
#     # Inicializar x con gradientes habilitados
#     x = torch.zeros(A.shape[1], requires_grad=True)
#     optimizer = torch.optim.SGD([x],lr=0.01, momentum=0.9)

#     for _ in range(100000):
#         optimizer.zero_grad()
#         constraint_penalty = torch.sum(torch.relu(A @ x - b))
#         objective = torch.dot(c, x) + 1000.0 * constraint_penalty
#         objective.backward(retain_graph=True)  # Mantén el grafo activo
#         optimizer.step()
#     return x

# solver = smooth_lp



# voy a probar con el solver del paper
def solver(c, A, b):
    ''' 
    usando el solver del paper
    '''
    return io.linprog(c, A, b)
    

In [15]:
plp_true(u_train[0])

[tensor([[ 0.8776],
         [-0.4794]], dtype=torch.float64, grad_fn=<StackBackward0>),
 tensor([[-1.0000,  0.0000],
         [ 0.0000, -1.0000],
         [ 1.0000,  0.5000]], dtype=torch.float64, grad_fn=<StackBackward0>),
 tensor([[-0.3000],
         [ 0.3000],
         [ 0.8500]], dtype=torch.float64, grad_fn=<StackBackward0>),
 None,
 None]

In [16]:
io.linprog(*plp_true(u_train[0]))

tensor([[0.3000],
        [1.1000]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [17]:
c = torch.tensor([[-1.0],
                  [-1.0]], dtype=torch.float64)

A = torch.tensor([[ 1.0000,  0.0000],
                  [ 0.0000,  1.0000]], dtype=torch.float64)

b = torch.tensor([[ 0.0],
                  [ 0.0]], dtype=torch.float64)

x = solver(c, A, b)

x

tensor([[-1.5257e-05],
        [-1.5257e-05]], dtype=torch.float64)

In [121]:
c =  torch.tensor([[-0.2775],
        [-0.2279]], dtype=torch.float64)
A =  torch.tensor([[ 0.2294, -0.0537],
        [-0.1553,  0.2169]], dtype=torch.float64)
b =  torch.tensor([[-0.2831],
        [ 0.3159]], dtype=torch.float64)

x = solver(c, A, b)
x

tensor([[-1.0730],
        [ 0.6881]], dtype=torch.float64)

Esta version es la red neuronal en la que la resolucion del problema de optimizacion esta dentro de la red

In [199]:
# definimos la red (hay que revisar la forma de la red y el por qué)
class ParametricLPNet(nn.Module):
    def __init__(self):
        super(ParametricLPNet, self).__init__()
        # Entrada de dimensión 1, salida 8 (2 para c, 4 para A, 2 para b)
        self.fc = nn.Sequential(
            nn.Linear(1, 8),
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 8)  # c (2), A (4), b (2)
        )

    def forward(self, u):
        output = self.fc(u)
        # c es un vector 2x1, A es una matriz 2x2, b es un vector 2x1
        
        # c = output[:, 0:2].t().to(dtype=torch.float64)
        # A = output[:, 2:6].reshape(-1, 2, 2).to(dtype=torch.float64)
        # b = output[:, 6:8].t().to(dtype=torch.float64)
        return torch.stack([solver(c[i].t().to(dtype=torch.float64), A[i].reshape(2, 2).to(dtype=torch.float64), b[i].t().to(dtype=torch.float64)) for i in range(u.shape[0])]) # Resolver LPs ver si hay alguna manera mas eficiente de hacer esto


In [200]:
loss_fn = nn.MSELoss()

In [201]:
# Crear la red neuronal
model = ParametricLPNet()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # elegir una de las dos

In [202]:
# para ver el modelo
model

ParametricLPNet(
  (fc): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
  )
)

In [203]:
# inicializamod los pesos de la red 
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='leaky_relu') # inicializamos los pesos de la red con Kaiming
        nn.init.zeros_(m.bias) # inicializamos los bias a 0

initialize_weights(model)

In [204]:
def near(rs, x_batch, tol=1e-3):
    '''
    rs: tensor formado por las soluciones de los LPs
    x_batch: tensor formado por las soluciones deseadas de los LPs
    tol: tolerancia para considerar que dos vectores son iguales

    Esta función calcula el número de soluciones de los LPs que están 
    a una distancia menor que tol de las soluciones deseadas.
    '''
    matriz_diferencias = rs - x_batch # matriz de diferencias entre las soluciones de los LPs y las soluciones deseadas
    matriz_al_cuadrado = matriz_diferencias ** 2 # elevo cada elemento al cuadrado
    distancias = torch.sum(matriz_al_cuadrado, dim=1) # sumo los elementos de cada fila
    return torch.sum(distancias < tol).item()

In [205]:
tol = 0.1

def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    loss_media = 0.0
    accuracy = 0.0
    batch_size = dataloader.batch_size
    for batch, (u_batch, x_batch) in enumerate(dataloader):

        rs = model(u_batch)
        loss = loss_fn(rs, x_batch) # Calcular la pérdida

        # Backpropagation y optimización
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 8 == 0: # cambiar este numero para que salga cada cierto numero de iteraciones
            loss = loss.item()
            current = batch * batch_size + batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
        loss_media += loss
        accuracy += near(rs, x_batch, tol)

    return loss_media / num_batches, accuracy / size

In [206]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size=len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #with torch.no_grad():
    for u_batch, x_batch in dataloader:
        rs = model(u_batch)
        test_loss += loss_fn(rs, x_batch).item()
        correct += near(rs, x_batch, tol) # consideramos correcto si se acerca a la solucion en la distancia euclidea 
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n\n")

In [207]:
# Entrenamiento
epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss, accuracy = train_loop(dataloader, model, loss_fn, optimizer)
    print(f"\nTraining Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {loss:>8f} \n")
    
    test_loop(val_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: shape '[2, 2]' is invalid for input of size 2

In [134]:
torch.save(model.state_dict(), "model_weights.pth")

In [135]:
model(torch.tensor([[1.5]]))

c =  tensor([[ 0.1356],
        [-0.1666]], dtype=torch.float64, grad_fn=<ToCopyBackward0>)
A =  tensor([[ 0.0897, -0.1552],
        [ 0.0063,  0.2630]], dtype=torch.float64, grad_fn=<ToCopyBackward0>)
b =  tensor([[0.2870],
        [0.0503]], dtype=torch.float64, grad_fn=<ToCopyBackward0>)


UnboundedConstraintsError: 

In [ ]:
u_test = torch.tensor(np.linspace(0.1, 4, 16).reshape((-1, 1)), dtype=torch.float64)
u_test

tensor([[0.1000],
        [0.3600],
        [0.6200],
        [0.8800],
        [1.1400],
        [1.4000],
        [1.6600],
        [1.9200],
        [2.1800],
        [2.4400],
        [2.7000],
        [2.9600],
        [3.2200],
        [3.4800],
        [3.7400],
        [4.0000]], dtype=torch.float64)

ahora probamos como funciona para un x nuevo

In [ ]:
x_test = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_test])
x_test

tensor([[-0.0200,  0.0200],
        [-0.0720,  0.0720],
        [ 0.9123,  0.1240],
        [ 0.8604,  0.1760],
        [ 0.7993,  0.2280],
        [ 0.7293,  0.2800],
        [ 0.6503,  0.3320],
        [ 0.5622,  0.3840],
        [ 0.4652,  0.4360],
        [ 0.3591,  0.4880],
        [ 0.2440,  0.5400],
        [ 0.1199,  0.5920],
        [-0.0133,  0.6440],
        [-0.6960,  0.9463],
        [-0.7480,  0.9445],
        [-0.8000,  0.9429]], dtype=torch.float64)

In [ ]:
model.eval()

ParametricLPNet(
  (fc): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
  )
)

In [ ]:
u = torch.tensor([[0.25]], dtype=torch.float64)
x = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u])
u,x

(tensor([[0.2500]], dtype=torch.float64),
 tensor([[-0.0500,  0.0500]], dtype=torch.float64))

In [ ]:
model(torch.tensor([[0.25]]))

RuntimeError: Tensors must have same number of dimensions: got 1 and 2

In [ ]:
x_pred = model(torch.tensor([[0.25]]))
x_pred

Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.


tensor([[0., 0.]], grad_fn=<StackBackward0>)

In [ ]:
x, x_pred, torch.norm(x - x_pred)

(tensor([[-0.0500,  0.0500]], dtype=torch.float64),
 tensor([[0., 0.]], grad_fn=<StackBackward0>),
 tensor(0.0707, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>))

Revisar:
- Función de pérdida ✔️
- Función smooth_lp (a ver si puedo sustituir la funcion smooth_lp por la funcion de linprog de deep_inv_opt)
- Organizar y explicar el código
- Estudiar por qué no disminuye el loss cuando entreno el modelo: es porque está mostrando el loss en el conjunto de test y no en el de entrenamiento ✔️
- Entrenar el modelo con más épocas y datos
- Pensar qué optimizador es mejor ✔️
- Pensar qué estructura de la red es mejor ✔️
- Inicialización de los pesos de la red ✔️
- Arreglar el porcentaje de aciertos ✔️



Bitácora: ahora lo que pasa es que da error porque el problema de optimización no está acotado, lo que tengo que hacer es revisar en el paper el intervalo donde se mueve la u porque creo que lo he puesto mal. Cuando arregle esto, tengo que volver a entrenar el modelo y ver si se está entrenando bien. También tengo que probar resolviendo el problema de optimización con la función de linprog de deep_inv_opt y con la de cvxpy a ver si alguna de las dos funciona mejor.